# Parse Q Data
Start by importing some utilities and the raw data:

In [13]:
import rawData, json, jupyterImport
jupyterImport.enableJupyterImports()
from Utilities import find, genSamples, genSample, pluck, copyKeys, combineDicts

In [14]:
def extractRawReports(data=rawData.q()):
  rawReportsDict = combineDicts(map(lambda s: s['reports'], data))
  for reportId, rawReport in rawReportsDict.iteritems():
    rawReport['reportId'] = reportId
  return rawReportsDict.values()

For some reason, Q reports don't include term / year anywhere on the pages. We have to scrape this data separately and then merge it with the reports:

In [15]:
def addTermYears(reports):
  origins = {}
  for key, termDict in rawData.qCourseIds().iteritems():
    for deptList in termDict.values():
      for courseId in deptList:
        term = key.split('_')[1]
        year = key.split('_')[0]
        term = 'Fall' if term == '1' else 'Spring'
        year = year if term == 'Fall' else str(int(year) + 1)
        origins[courseId] = {'term': term, 'year': year}
  for report in reports:
    if report['reportId'] in origins:
      report.update(origins[report['reportId']])

In [16]:
import q_map
def parseReportsAndComments(rawReports):
  comments = {}
  reports = []
  for rawReport in rawReports:
    report = {}
    for mapper in q_map.mappers:
      mapper.map(rawReport, to=report)
    reports.append(report)
    q_map.comments.map(rawReport, to=comments)
  addTermYears(reports)
  return (reports, comments)

In [17]:
reports, comments = parseReportsAndComments(extractRawReports())

In [ ]:
### Run to inspect
genSample(reports)